In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import Dataset
import gradio as gr

data = [
    {"text": "Riddle: What number becomes zero when you subtract 15 from half of it?\nAnswer: 30"},
    {"text": "Riddle: I am a number that when doubled and then reduced by 20 gives 40.\nAnswer: 30"},
    {"text": "Riddle: If you add 10 to a number and then subtract 5, you get 25.\nAnswer: 20"},
    {"text": "Riddle: I am 15 less than twice my value.\nAnswer: 15"},
    {"text": "Riddle: A number when halved and then increased by 10 becomes 25.\nAnswer: 30"},
    {"text": "Riddle: When you multiply a number by 3 and subtract 9, the result is 18.\nAnswer: 9"},
    {"text": "Riddle: If a number is decreased by 8 and then doubled, you get 14.\nAnswer: 15"},
    {"text": "Riddle: A number when tripled and then increased by 5 equals 20.\nAnswer: 5"},
    {"text": "Riddle: When you add 7 to half of a number, you get 19.\nAnswer: 24"},
    {"text": "Riddle: A number is increased by 9 and then halved to get 15.\nAnswer: 21"},
    {"text": "Riddle: When you subtract 4 from a number and then multiply by 3, the result is 33.\nAnswer: 15"},
    {"text": "Riddle: A number reduced by 6 equals one-third of itself.\nAnswer: 9"},
    {"text": "Riddle: When you double a number and add 10, you get 30.\nAnswer: 10"},
    {"text": "Riddle: A number, when 5 is subtracted and then multiplied by 2, gives 20.\nAnswer: 15"},
    {"text": "Riddle: If a number is multiplied by 4 and then decreased by 8, the result is 24.\nAnswer: 8"},
    {"text": "Riddle: A number, when divided by 2 and then increased by 7, equals 17.\nAnswer: 20"},
    {"text": "Riddle: When you subtract 3 from a number and then square the result, you get 49.\nAnswer: 10"},
    {"text": "Riddle: If 12 is added to a number, the result is three times the number.\nAnswer: 6"},
    {"text": "Riddle: A number increased by 50% equals 27.\nAnswer: 18"},
    {"text": "Riddle: If a number is halved and then 4 is subtracted, the result is 8.\nAnswer: 24"},
    {"text": "Riddle: A number, when 2 is added, becomes twice the original number.\nAnswer: 2"},
    {"text": "Riddle: When you triple a number and subtract 7, the result is 14.\nAnswer: 7"},
    {"text": "Riddle: A number, when reduced by 2 and then divided by 4, gives 5.\nAnswer: 22"},
    {"text": "Riddle: When you add 8 to a number and then multiply by 2, you get 40.\nAnswer: 12"},
    {"text": "Riddle: A number, when doubled, is 16 more than the number itself.\nAnswer: 16"},
    {"text": "Riddle: A number that is increased by 3 and then multiplied by 2 equals 26.\nAnswer: 10"},
    {"text": "Riddle: A number when reduced by 4 and then doubled equals 12.\nAnswer: 10"},
    {"text": "Riddle: If you subtract 2 from a number and then double it, you get 14.\nAnswer: 9"},
    {"text": "Riddle: A number when tripled and decreased by 5 equals 16.\nAnswer: 7"},
    {"text": "Riddle: If you add 5 to a number and then double the result, you get 30.\nAnswer: 10"}
]

dataset = Dataset.from_list(data)

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(["text"])  # remove raw text column if not needed
tokenized_dataset.set_format("torch")

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))  # Adjust for the added pad token

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="./gpt2-math-riddle",
    overwrite_output_dir=True,
    num_train_epochs=15,                    # Increased epochs
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,          # Simulate a larger batch size
    learning_rate=3e-5,                     # Lower learning rate
    weight_decay=0.01,                      # Optional: add weight decay
    warmup_steps=100,                       # Optional: add warmup steps
    save_steps=500,
    save_total_limit=2,
    logging_steps=50,
    prediction_loss_only=True,
    report_to=[]                           # Disable wandb logging
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

trainer.train()

model.eval()

# Update model config for pad_token_id if not already set
model.config.pad_token_id = tokenizer.eos_token_id

# Gradio UI for testing
def generate_riddle(prompt):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        input_ids,
        max_length=35,             # Further limit the output length
        do_sample=True,            # Enable sampling
        top_k=50,                  # Top-k sampling
        top_p=0.92,                # Nucleus sampling
        temperature=0.5,           # Lower temperature for more deterministic outputs
        repetition_penalty=1.2,    # Penalize repetition
        no_repeat_ngram_size=3,    # Prevent 3-gram repetition
        num_return_sequences=5,
        pad_token_id=tokenizer.eos_token_id
    )

    generated_texts = []
    for output in outputs:
        generated_text = tokenizer.decode(output, skip_special_tokens=True)
        if "\nAnswer:" in generated_text:
            parts = generated_text.split("\nAnswer:")
            answer_part = parts[1].split('.')[0] + "."
            generated_text = parts[0] + "\nAnswer:" + answer_part
        generated_texts.append(generated_text)

    return generated_texts

iface = gr.Interface(fn=generate_riddle, inputs="text", outputs="text", title="Math Riddle Generator", description="Enter a prompt to generate a riddle.")
iface.launch()



In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model
from datasets import Dataset
import gradio as gr  # Import Gradio for UI

# Define a dataset of 20 incorrect math memes with corrections and explanations.
data = [
    {"text": "Incorrect: 8 ÷ 2(2+2) = 1? Correct: 8 ÷ 2(2+2) = 16. Explanation: Evaluate parentheses first then perform division and multiplication sequentially."},
    {"text": "Incorrect: 5 + 5 = 20? Correct: 5 + 5 = 10. Explanation: Simple addition error."},
    {"text": "Incorrect: 6 * 6 = 36 but 6 / 6 = 6? Correct: 6 / 6 = 1. Explanation: A number divided by itself equals 1."},
    {"text": "Incorrect: 2^3 = 6? Correct: 2^3 = 8. Explanation: 2 cubed is 8."},
    {"text": "Incorrect: √16 = 5? Correct: √16 = 4. Explanation: The square root of 16 is 4."},
    {"text": "Incorrect: 9 - 3 = 3? Correct: 9 - 3 = 6. Explanation: Correct subtraction yields 6."},
    {"text": "Incorrect: 4 * 4 = 8? Correct: 4 * 4 = 16. Explanation: Multiplication error."},
    {"text": "Incorrect: 10 / 2 = 10? Correct: 10 / 2 = 5. Explanation: Division error."},
    {"text": "Incorrect: 15% of 200 = 50? Correct: 15% of 200 = 30. Explanation: 15% of 200 equals 30."},
    {"text": "Incorrect: 100 / 4 = 20? Correct: 100 / 4 = 25. Explanation: Division error."},
    {"text": "Incorrect: 3 + 7 = 11? Correct: 3 + 7 = 10. Explanation: 3 plus 7 equals 10."},
    {"text": "Incorrect: 2 * 3 + 4 = 14? Correct: 2 * 3 + 4 = 10. Explanation: Follow order of operations: multiply then add."},
    {"text": "Incorrect: 12 / 3 * 2 = 10? Correct: 12 / 3 * 2 = 8. Explanation: 12 divided by 3 is 4; 4 times 2 is 8."},
    {"text": "Incorrect: 7 * 7 = 42? Correct: 7 * 7 = 49. Explanation: Multiplication error."},
    {"text": "Incorrect: 14 - 7 = 8? Correct: 14 - 7 = 7. Explanation: Subtraction error."},
    {"text": "Incorrect: (3 + 2) * 2 = 12? Correct: (3 + 2) * 2 = 10. Explanation: Add first, then multiply."},
    {"text": "Incorrect: 50% of 100 = 60? Correct: 50% of 100 = 50. Explanation: 50% is half of 100."},
    {"text": "Incorrect: 9 + 9 = 18 then 18 / 2 = 10? Correct: 18 / 2 = 9. Explanation: Division error."},
    {"text": "Incorrect: 5! = 100? Correct: 5! = 120. Explanation: 5 factorial is 120."},
    {"text": "Incorrect: 3^2 + 4^2 = 14? Correct: 3^2 + 4^2 = 25. Explanation: 9 + 16 equals 25."}
]

# Convert the list to a Hugging Face Dataset.
dataset = Dataset.from_list(data)
print("Dataset created with", len(dataset), "examples.")

# Load the GPT-2 tokenizer and model.
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# GPT-2 does not have an official pad token; use the eos_token.
tokenizer.pad_token = tokenizer.eos_token

model = GPT2LMHeadModel.from_pretrained("gpt2")

# Configure LoRA for efficient fine-tuning.
lora_config = LoraConfig(
    task_type="CAUSAL_LM",  # For language modeling.
    r=8,
    lora_alpha=32,
    lora_dropout=0.1
)

# Wrap the model with LoRA.
model = get_peft_model(model, lora_config)
print("Model loaded and LoRA configured.")

# Tokenize each example.
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, max_length=128, padding="max_length")

tokenized_dataset = dataset.map(tokenize_function, batched=False)
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])
print("Dataset tokenized.")

training_args = TrainingArguments(
    output_dir="output",
    per_device_train_batch_size=1,
    num_train_epochs=5,  # Increase epochs to help the model learn from 20 examples.
    logging_steps=1,
    save_strategy="epoch",
    learning_rate=3e-5,  # Slightly lower learning rate.
    weight_decay=0.01,
    report_to="none"
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator
)

print("Starting training...")
trainer.train()
print("Training complete!")

# Gradio UI for testing the model
def correct_math(prompt):
    model.eval()  # Set model to evaluation mode.
    inputs = tokenizer(prompt, return_tensors="pt", padding=True)
    input_ids = inputs.input_ids.to(model.device)
    attention_mask = inputs.attention_mask.to(model.device)

    outputs = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_new_tokens=50,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.90,
        pad_token_id=tokenizer.eos_token_id
    )

    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result

# Create Gradio interface
gr.Interface(fn=correct_math, inputs="text", outputs="text", title="Math Correction Model", description="Enter an incorrect math statement to get the correct answer and explanation.").launch()



In [ ]:
from transformers import GPT2LMHeadModel, TrainingArguments, Trainer, DataCollatorForLanguageModeling, GPT2Tokenizer
import gradio as gr

# Create a list of 30 emoji math problems with their solutions.
# Format: "Q: [emoji math equation]\nA: [solution]"
data = [
    "Q: 🍎 + 🍎 + 🍎 = 12\nA: 4",
    "Q: 🎲 + 🎲 = 12\nA: 6",
    "Q: 🚗 + 🚗 + 🚗 + 🚗 = 20\nA: 5",
    "Q: 🍌 + 🍌 + 🍌 + 🍌 + 🍌 = 15\nA: 3",
    "Q: 🍓 + 🍓 = 8\nA: 4",
    "Q: 🍕 + 🍕 + 🍕 = 18\nA: 6",
    "Q: 🍩 + 🍩 + 🍩 + 🍩 = 20\nA: 5",
    "Q: 🌟 + 🌟 + 🌟 = 9\nA: 3",
    "Q: 🎈 + 🎈 = 14\nA: 7",
    "Q: 🎂 + 🎂 + 🎂 = 15\nA: 5",
    "Q: 🍪 + 🍪 + 🍪 + 🍪 = 16\nA: 4",
    "Q: 🍭 + 🍭 + 🍭 = 15\nA: 5",
    "Q: 🧁 + 🧁 = 10\nA: 5",
    "Q: 🥑 + 🥑 + 🥑 = 12\nA: 4",
    "Q: 🍇 + 🍇 = 10\nA: 5",
    "Q: 🍒 + 🍒 + 🍒 = 15\nA: 5",
    "Q: 🍍 + 🍍 = 14\nA: 7",
    "Q: 🍉 + 🍉 + 🍉 + 🍉 = 20\nA: 5",
    "Q: 🥭 + 🥭 = 16\nA: 8",
    "Q: 🍈 + 🍈 + 🍈 = 9\nA: 3",
    "Q: 🍑 + 🍑 + 🍑 + 🍑 = 20\nA: 5",
    "Q: 🍏 + 🍏 = 10\nA: 5",
    "Q: 🍋 + 🍋 + 🍋 = 12\nA: 4",
    "Q: 🍊 + 🍊 = 10\nA: 5",
    "Q: 🥝 + 🥝 + 🥝 = 15\nA: 5",
    "Q: 🍐 + 🍐 = 8\nA: 4",
    "Q: 🍆 + 🍆 + 🍆 + 🍆 = 16\nA: 4",
    "Q: 🥕 + 🥕 = 10\nA: 5",
    "Q: 🌽 + 🌽 + 🌽 = 9\nA: 3",
    "Q: 🥔 + 🥔 + 🥔 + 🥔 = 20\nA: 5"
]

# For training with Hugging Face's datasets, we create a dictionary.
from datasets import Dataset
dataset = Dataset.from_dict({"text": data})

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, max_length=128, padding="max_length")

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

# Load GPT-2 model
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.config.pad_token_id = tokenizer.eos_token_id

# Create a data collator for language modeling that will handle padding dynamically
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./emoji_math_model",
    overwrite_output_dir=True,
    num_train_epochs=8,
    per_device_train_batch_size=4,
    save_steps=50,
    save_total_limit=2,
    logging_steps=10,
    learning_rate=1e-5
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

# Start training
trainer.train()

import logging
logging.getLogger("transformers").setLevel(logging.ERROR)  # Suppress transformer warnings

import re

def generate_single_answer(prompt, max_new_tokens=10):
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to(model.device)
    attention_mask = inputs["attention_mask"].to(model.device)

    output = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_new_tokens=max_new_tokens,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=False,
        repetition_penalty=2.0
    )

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

    if prompt in generated_text:
        generated_text = generated_text.split(prompt, 1)[1]

    match = re.search(r'\b(\d+)\b', generated_text)
    if match:
        answer = match.group(1)
    else:
        answer = generated_text.strip()

    return answer

# Gradio UI
def gradio_interface(prompt):
    return generate_single_answer(prompt)

iface = gr.Interface(fn=gradio_interface, inputs="text", outputs="text", title="Emoji Math Solver", description="Enter an emoji math problem to get the answer.")
iface.launch()
